Author: Ashkan Nikfarjam
basic info from "https://en.wikipedia.org/wiki/Healthcare_in_Germany"
Germans are offered three mandatory health benefits, which are co-financed by employer and employee: health insurance, accident insurance, and long-term care insurance
list of hospitals in germany: "https://en.wikipedia.org/wiki/List_of_hospitals_in_Germany"

In [2]:
from bs4 import BeautifulSoup
import requests
url = "https://en.wikipedia.org/wiki/List_of_hospitals_in_Germany"
response = requests.get(url)
if response.status_code == 200:
    health_soup = BeautifulSoup(response.content, 'html.parser')
else:
    print('couldnt reach', url)

now we have everything lets write everything in and html file and take a peack

In [3]:
with open("hospitals.html", 'w') as file:
    file.write(health_soup.prettify())

In [4]:
rows = health_soup.find_all('tr')
#now i want to exract all the hospilat names, city and number of beds
# Extract data from each row
hospital_lst = []
city_lst = []
beds_lst = []
for row in rows:
    # Find all table data elements in the row
    cells = row.find_all('td')
    
    # Extract hospital name, city, number of beds
    if len(cells) >= 3:  # Ensure the row has enough data cells
        hospital = cells[0].get_text(strip=True)
        city = cells[1].get_text(strip=True)
        beds = cells[2].get_text(strip=True)
        
        # Remove unwanted characters or tags from beds
        beds = beds.split('[')[0]  # Remove citation references
        
        # store the extracted data
        #print(f"Hospital: {hospital}, City: {city}, Beds: {beds}")
        hospital_lst.append(hospital)
        city_lst.append(city)
        beds_lst.append(beds)

Now that i extracted the data i needed lets create a data frame to represent these

In [5]:
print(len(hospital_lst))
print(len(city_lst))
print(len(beds_lst))

111
111
111


In [6]:
import pandas as pd

health_df = pd.DataFrame({'Hospital Name': hospital_lst, 'City': city_lst, 'Number of beds': beds_lst})
health_df.head()

,Hospital Name,City,Number of beds
0,Charité – Universitätsmedizin Berlin,Berlin,"3,011"
1,University Hospital Heidelberg,Heidelberg,"1,991"
2,Klinikum der Universität München,Munich,"2,050"
3,Vivantes Hospital Group,Berlin,"5,856"
4,University Medical Center Hamburg-Eppendorf,Hamburg,"1,436"


now we can call this our raw dataframe to use these info for visualization and ML model training one thing could be done is to goup data by city aggregate number of beds and also count the number of hospitals

In [7]:
agg_df = health_df.copy()
numBeds_hsptl_df = agg_df.groupby(by='City').agg({agg_df.columns[2]:'sum'})
numBeds_hsptl_df.head()

,Number of beds
City,
Aachen,"1,517"
Amberg,588
Augsburg,"1,6991,741"
Bad Berka,728
Bad Oeynhausen,240104


In [8]:
numHsptl = agg_df.City.value_counts()
numHsptl_df = pd.DataFrame(numHsptl).reset_index()
numHsptl_df.head()

,City,count
0,Berlin,9
1,Hamburg,9
2,Kassel,5
3,Munich,5
4,Cologne,4


In [9]:
merged_data = pd.merge(numBeds_hsptl_df, numHsptl_df, on='City')
merged_data.rename(columns={'count':'Number of Hospitals'}, inplace=True)
merged_data.head()

,City,Number of beds,Number of Hospitals
0,Aachen,"1,517",1
1,Amberg,588,1
2,Augsburg,"1,6991,741",2
3,Bad Berka,728,1
4,Bad Oeynhausen,240104,2


Now i can conver this data frame to csv so i could use the data in my UI file

In [10]:
merged_data.to_csv("numberofHospitals.csv", index=False)

Pharmecies are another set of data to look at here source: https://www.govdata.de/web/guest/suchen/-/details/kreis-viersen-poi-osm-nrw-apotheken-tankstellen77d2f


In [38]:
from googletrans import Translator

pharmacy = pd.read_csv("apotheken_nrw.csv")
pharmacy = pharmacy[['name','adresse']]
pharmacy.adresse = pharmacy.adresse.str.split(',').str[-1]
pharmacy_df = pd.DataFrame(pharmacy.adresse.value_counts()).reset_index()
pharmacy_df.rename(columns={'adresse':'city', 'count':'number of pharmecies'}, inplace=True)
pharmacy_df.head()



,city,number of pharmecies
0,-,1189
1,Düsseldorf,103
2,Essen,94
3,Bonn,86
4,Bochum,80


In [40]:
from fuzzywuzzy import process
german_city = pd.read_csv("../German cities.csv")
#german_city.head()
real_city_lst = german_city.city.to_list()
#real_city_lst
def find_match(city, choices):
    return process.extractOne(city, choices)[0]
test_pd = pharmacy_df.copy()
test_pd['city']= test_pd.city.apply(lambda x: find_match(x, real_city_lst))

test_pd.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,city,number of pharmecies
0,Berlin,1189
1,Düsseldorf,103
2,Essen,94
3,Bonn,86
4,Bochum,80


In [39]:
test_pd.to_csv("numberOfPharmecies.csv", index=False)